# Torch vision example

Source: https://www.learnopencv.com/pytorch-for-beginners-image-classification-using-pre-trained-models/
with minor modifications. Errors are likely mine.

In [ ]:
from torchvision import models
import torch

# Let us look at the Deep learning architectures implemented in the torch vision library.
dir(models)

Notice that there is one entry called **AlexNet** and one called **alexnet**. The capitalised name refers to the Python class (AlexNet) whereas alexnet is a convenience function that returns the model instantiated from the AlexNet class.

In [ ]:
from torchvision.models.alexnet import AlexNet_Weights

alexnet = models.alexnet(weights=AlexNet_Weights.DEFAULT)
print(alexnet)

In [ ]:
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt


In [ ]:
from torchvision import transforms
import json

def imagenet_class(idx: int, cache=None):
    if cache is None:
        cache = json.load(open(Path('./imagenet_classes.json')))["imagenet_classes"]
        assert len(cache) == 1000 and cache[11] == 'goldfinch'

    return cache[idx]

transform = transforms.Compose([        # Defining a variable transforms
   transforms.Resize(256),                # Resize the image to 256×256 pixels
   transforms.CenterCrop(224),            # Crop the image to 224×224 pixels about the center
   transforms.ToTensor(),                 # Convert the image to PyTorch Tensor data type
   transforms.Normalize(                  # Normalize the image
   mean=[0.485, 0.456, 0.406],            # Mean and std of image as also used when training the network
   std=[0.229, 0.224, 0.225]
 )])

def evaluate_image(img):
    img_t = transform(img)                # Apply the transformations on the image
    batch_t = torch.unsqueeze(img_t, 0)   # Add a batch dimension to the image
    alexnet.eval()                        # Set the network to evaluation mode
    out = alexnet(batch_t)                # Forward propagate the image

    _, indices = torch.sort(out, descending=True)
    percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100
    return [(imagenet_class(idx), percentage[idx].item()) for idx in indices[0][:5]]

In [ ]:
img = Image.open(Path('./dog.jpg'))
best = evaluate_image(img)
best

# running testing alexnet with stl10 torchvision dataset

As it happens, the INaturalist dataset was good for our purposes but way too big.
As [the STL-10 page notes](https://cs.stanford.edu/~acoates/stl10/) these
images were drawn from ImageNet so presumably are readily recognized.

In [ ]:
from charmory.evaluation import SysConfig
from torchvision.datasets import STL10
from pprint import pprint

root = SysConfig().dataset_cache / 'stl10'
stl = STL10(root=root, split='test', download=True)

stl_classes = 'airplane, bird, car, cat, deer, dog, horse, monkey, ship, truck'.split(', ')


for i in range(10):
    img, label = stl[i]
    target = stl_classes[label]
    plt.imshow(img)
    plt.show()
    print(f"{label=}, {target=}")
    best = evaluate_image(img)
    pprint(best)

Well that didn't work. There are only 10 classes in STL-10 and the model outputs
1000 classes. As shown in the first output from the cell above, the first STL item
is a horse, but imagenet has no horse class. So we get the most likely as "oxcart"
which isn't a bad guess, but it is not a horse.


# using Imagenet-1k dataset

The Imagenet-1k dataset in torchvision requires tarfiles downloaded from image-net.org,
but I've applied to get access and it takes "up to 5 days". So I've downloaded parquet
files for the same dataset and have stashed them on our 
s3://armory-library-data/datasets/huggingface/imagenet/ so at least I don't have to
hunt them down again.

This means that I'm going to use a huggingface dataloader for now.

In [ ]:
import pyarrow.parquet as pq
dataset_local = str(SysConfig().dataset_cache / 'imagenet-1k/validation_split')
imagenet = pq.ParquetDataset(dataset_local)
table = imagenet.read()


In [ ]:
for x in table:
    print(x)
    break

In [ ]:
target = table['label'][0]
img_data = table['image'][0]
print(f"{target=} {type(img_data)=}")
print(f"{img_data.type=} {img_data.keys=}")